In [ ]:
import syft as sy
import torch
from tools import models
import numpy as np
import random
import pandas as pd

sy.load('opacus')
np.random.seed(42) # The meaning of life!

In [ ]:
duet = sy.join_duet(loopback=True)

In [ ]:
# Getting the pionters to the data
train_data_ptr = duet.store[0]
train_labels_ptr = duet.store[1]

test_data_ptr = duet.store[2]
test_labels_ptr = duet.store[3]

val_data_ptr = duet.store[4]
val_labels_ptr = duet.store[5]

val_data = val_data_ptr.get(request_block=True, reason="Needed for Validation!")
val_labels = val_labels_ptr.get(request_block=True, reason="Needed for Validation!")

In [ ]:
# Parameters for training and Differential Privacy
length = len(train_data_ptr)
BATCH_SIZE = 50
EPOCHS = 20
SAMPLE_SIZE = length - length % BATCH_SIZE # NOTE: Current implementation only trains data in multiples of batch size. So BATCH_SIZE % LENGTH amount of data will not be used for training.
SAMPLE_RATE = BATCH_SIZE / SAMPLE_SIZE
DELTA = 0.001 # Set to be less then the inverse of the training dataset (from https://opacus.ai/tutorials/building_image_classifier)
NOISE_MULTIPLIER = 2.0 # The amount of noise sampled and added to the average of the gradients in a batch (from https://opacus.ai/tutorials/building_image_classifier)
MAX_GRAD_NORM = 1.2 # The maximum L2 norm of per-sample gradients before they are aggregated by the averaging step (from https://opacus.ai/tutorials/building_image_classifier)

DP = True

# Getting remote and local instances
local_model = models.Deep2DNet(torch)
remote_model = local_model.send(duet)
remote_torch = duet.torch
remote_opacus = duet.opacus

# Setting device to train on
cuda_available = remote_torch.cuda.is_available().get(request_block=True, reason='Need to check for available GPU!')
if cuda_available:
    device = remote_torch.device('cuda:0')
    remote_model.cuda(device)
else:
    device = remote_torch.device('cpu')
    remote_model.cpu()

# Optimizer and Loss Function
params = remote_model.parameters()
optim = remote_torch.optim.Adam(params=params, lr=0.003) # without DP: 0.0001 // with DP: 0.002-0.003
loss_function = remote_torch.nn.CrossEntropyLoss()

# Setting up Differential Privacy Engine
if DP:
    privacy_engine_ptr = remote_opacus.privacy_engine.PrivacyEngine(
        remote_model.real_module, sample_rate=SAMPLE_RATE,
        noise_multiplier=NOISE_MULTIPLIER, max_grad_norm=MAX_GRAD_NORM
    )
    privacy_engine_ptr.attach(optim)
else:
    privacy_engine_ptr = None


In [ ]:
import time

def train(batch_size, epochs, model, 
          torch_ref, optim, loss_function, 
          train_data, train_labels, test_data, 
          test_labels, input_shape, device, privacy_engine=None):
    
    # Variables to track
    losses = [] # Average loss per epoch
    test_accs = []
    test_losses = []
    epsilons = [] 
    alphas = []
    epoch_times = [] # Training times for each epoch
    
    # Divide dataset into batches (sadly remote DataLoaders aren't yet a thing in pysyft)
    length = len(train_data)
    
    if length % batch_size != 0:
        cut_data = train_data[:length - length % batch_size]
        cut_labels = train_labels[:length - length % batch_size]
        
    shape = [-1, batch_size]
    shape.extend(input_shape)
    
    batch_data = cut_data.view(shape)
    batch_labels = cut_labels.view(-1, batch_size)
    
    # Prepare indices for randomization of order for each epoch
    indices = np.arange(int(length / batch_size))
    
    for epoch in range(epochs):
        epoch_start = time.time()
        epoch_loss = []
        
        model.train()
        
        np.random.shuffle(indices)
        
        print(f'###### Epoch {epoch + 1} ######')
        for i in indices:
            optim.zero_grad()
            
            output = model(batch_data[int(i)].to(device))
            
            loss = loss_function(output, batch_labels[int(i)].to(device))
            loss_item = loss.item()
            
            if model.is_local:
                loss_value = loss_item
            else:
                loss_value = loss_item.get(reason="To evaluate training progress", request_block=True, timeout_secs=5)
            print(f'Training Loss: {loss_value}')
            epoch_loss.append(loss_value)
        
            loss.backward()
            optim.step()
        
        # Cheking our privacy budget
        if privacy_engine is not None:
            epsilon_tuple = privacy_engine.get_privacy_spent(DELTA)
            epsilon_ptr = epsilon_tuple[0].resolve_pointer_type()
            best_alpha_ptr = epsilon_tuple[1].resolve_pointer_type()

            epsilon = epsilon_ptr.get(
                reason="So we dont go over it",
                request_block=True,
                timeout_secs=5
            )
            best_alpha = best_alpha_ptr.get(
                reason="So we dont go over it",
                request_block=True,
                timeout_secs=5
            )
            if epsilon is None:
                epsilon = float("-inf")
            if best_alpha is None:
                best_alpha = float("-inf")
            print(
                f"(ε = {epsilon:.2f}, δ = {DELTA}) for α = {best_alpha}"
            )
            epsilons.append(epsilon)
            alphas.append(best_alpha)
    
        test_acc, test_loss = test(model, loss_function, torch_ref, test_data, test_labels, device)
        print(f'Test Accuracy: {test_acc} ---- Test Loss: {test_loss}')
        
        epoch_end = time.time()
        print(f"Epoch time: {int(epoch_end - epoch_start)} seconds")
        
        losses.append(epoch_loss / int(length / batch_size))
        epoch_times.append(int(epoch_end - epoch_start))
        test_accs.append(test_acc)
        test_losses.append(test_loss)
        
    return losses, test_accs, test_losses, epsilons, alphas, epoch_times
                   
            
def test(model, loss_function, torch_ref, data, labels, device):
    model.eval()
    
    data = data.to(device)
    labels = labels.to(device)
    length = len(data)
    
    with torch_ref.no_grad():
        output = model(data)
        test_loss = loss_function(output, labels)
        prediction = output.argmax(dim=1)
        total = prediction.eq(labels).sum().item()
        
    acc_ptr = total / length
    if model.is_local:
        acc = acc_ptr
        loss = test_loss
    else:
        acc = acc_ptr.get(reason="To evaluate training progress", request_block=True, timeout_secs=5)
        loss = test_loss.get(reason="To evaluate training progress", request_block=True, timeout_secs=5)

    return acc, loss

In [ ]:
losses, test_accs, test_losses, epsilons, alphas, epoch_times = train(BATCH_SIZE, EPOCHS, 
                                                                      remote_model, remote_torch,
                                                                      optim, loss_function, 
                                                                      train_data_ptr, train_labels_ptr, 
                                                                      test_data_ptr, test_labels_ptr, 
                                                                      [1, 64, 64], device, privacy_engine_ptr)

In [ ]:
# Evalutating the model locally with the validation data
eval_model = remote_model.get(request_block=True, reason="Needed for local evaluation!")

val_acc, val_loss = test(eval_model, loss_function, torch, val_data, val_labels, torch.device('cuda:0'))

print(f'Validation Accuracy: {val_acc} ---- Validation Loss: {val_loss}')

# length = len(val_data)

# val_data.to(torch.device('cuda:0'))
# val_labels.to(torch.device('cuda:0'))

# eval_model = remote_model.get(request_block=True, reason="Needed for local evaluation!")

# with torch.no_grad():
#     eval_output = eval_model(val_data)
#     test_loss = loss_function(eval_output, val_labels)
#     prediction = eval_output.argmax(dim=1)
#     total = prediction.eq(val_labels).sum().item()

# print(f'Validation Accuracy: {total / length}')